In [1]:
import os
import pandas as pd
import numpy as np
import random
random.seed(301)
import pickle
from collections import namedtuple
from random import sample
from sklearn.utils import shuffle
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.model_selection import cross_validate 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score 
from sklearn.inspection import permutation_importance

In [2]:
out_dir = "DTGpred_results"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

cv_dir = os.path.join(out_dir, "CV")
if not os.path.exists(cv_dir):
    os.makedirs(cv_dir)

In [3]:
### import features ###

In [4]:
# Depmap gene depencency data

DepMap_gene = pd.read_csv("../data/DepMap_CRISPR_proc.csv")
DepMap_gene.set_index("ensembl_peptide_id", inplace=True)
DepMap_gene.index.name=""
DepMap_gene.drop(columns=["Unnamed: 0", "hgnc_symbol"], inplace=True)
DepMap_gene.columns = DepMap_gene.columns.str.replace("-", "_")
DepMap_gene.sort_index(inplace=True)
DepMap_gene.head()

,ACH_000159,ACH_000246,ACH_000250,ACH_000262,ACH_000272,ACH_000313,ACH_000411,ACH_000433,ACH_000459,ACH_000600,ACH_000649,ACH_000684,ACH_000709,ACH_000907,ACH_001687,ACH_002189
,,,,,,,,,,,,,,,,
ENSP00000000233,0.014338,0.014540,0.012414,0.008893,0.010900,0.019440,0.012632,0.080683,0.014468,0.007162,0.010833,0.028121,0.010661,0.029793,0.031375,0.016150
ENSP00000000412,0.022938,0.002169,0.011734,0.015452,0.007020,0.097726,0.009703,0.016714,0.018487,0.013308,0.007422,0.014256,0.019731,0.007376,0.024463,0.071701
ENSP00000001008,0.066159,0.016908,0.025620,0.070442,0.014907,0.034068,0.035755,0.013399,0.016162,0.012158,0.018524,0.015110,0.045925,0.022223,0.019659,0.062133
ENSP00000001146,0.019598,0.005997,0.021080,0.010145,0.011347,0.078015,0.004898,0.072882,0.069126,0.017841,0.003298,0.016184,0.033829,0.014241,0.007453,0.018045
ENSP00000002125,0.074261,0.307553,0.095555,0.039562,0.078746,0.038826,0.090187,0.057175,0.028967,0.041280,0.057014,0.017107,0.169583,0.052054,0.041762,0.022726


In [5]:
# Depmap average PPI neighbour gene depencency

DepMap_neigh = pd.read_csv("../Feature_space_generation/outputs/DepMap_PPI_neighbourhood_avg_scores.csv", index_col=0)
DepMap_neigh.index.name = ""
DepMap_neigh.columns = DepMap_neigh.columns.str.replace("-", "_")
DepMap_neigh.columns = [col + "_neigh" for col in DepMap_neigh.columns]
DepMap_neigh.sort_index(inplace=True)
DepMap_neigh.head()

,ACH_000159_neigh,ACH_000246_neigh,ACH_000250_neigh,ACH_000262_neigh,ACH_000272_neigh,ACH_000313_neigh,ACH_000411_neigh,ACH_000433_neigh,ACH_000459_neigh,ACH_000600_neigh,ACH_000649_neigh,ACH_000684_neigh,ACH_000709_neigh,ACH_000907_neigh,ACH_001687_neigh,ACH_002189_neigh
,,,,,,,,,,,,,,,,
ENSP00000000233,0.171082,0.215106,0.197154,0.189971,0.185885,0.215511,0.196795,0.184588,0.155852,0.170477,0.205071,0.166553,0.178327,0.168349,0.167894,0.211558
ENSP00000000412,0.134784,0.150380,0.170224,0.176201,0.175756,0.177303,0.163193,0.116987,0.114870,0.138439,0.181393,0.135799,0.141866,0.136071,0.149511,0.173547
ENSP00000001008,0.233053,0.255674,0.275618,0.256321,0.251233,0.252424,0.264634,0.235042,0.180657,0.229439,0.258836,0.207163,0.213516,0.199228,0.259583,0.248095
ENSP00000001146,0.052361,0.047647,0.049796,0.070940,0.057188,0.070805,0.050063,0.045282,0.032558,0.059307,0.053806,0.040982,0.047981,0.039699,0.045858,0.084290
ENSP00000002125,0.215080,0.470105,0.165638,0.224671,0.184290,0.117651,0.148655,0.193716,0.086706,0.143801,0.158032,0.112050,0.109280,0.095357,0.178308,0.192414


In [6]:
# Network centrality scores

centrality_scores = pd.read_csv("../Feature_space_generation/outputs/centrality_scores.csv", index_col=0)
centrality_scores.index.name = ""
centrality_scores.columns = centrality_scores.columns.str.replace(" ", "_")

common_ids = centrality_scores.index.intersection(DepMap_neigh.index) #17880

centrality_scores = centrality_scores.loc[common_ids]
centrality_scores.sort_index(inplace=True)
centrality_scores.head()

,Degree_Centrality,Betweenness_Centrality,Eigenvector_Centrality,PageRank_Centrality
,,,,
ENSP00000000233,0.006994,0.000126,0.002508,0.000074
ENSP00000000412,0.007046,0.000098,0.002646,0.000073
ENSP00000001008,0.009481,0.000180,0.007643,0.000088
ENSP00000001146,0.004611,0.000052,0.000936,0.000053
ENSP00000002125,0.004507,0.000038,0.001366,0.000047


In [7]:
# Ratios of PPI neigbours belonging to a particular signature

network_neighbourhood = pd.read_csv("../Feature_space_generation/outputs/network_neighbourhood.csv", index_col=0)
network_neighbourhood.index.name = ""
network_neighbourhood.columns = [col + "_neigh" for col in network_neighbourhood.columns]

network_neighbourhood = network_neighbourhood.loc[common_ids]
network_neighbourhood.sort_index(inplace=True)
network_neighbourhood.head()

,B_cells_markers_neigh,CAGs_neigh,CD4_markers_neigh,CD8_markers_neigh,Clark_CNV_GAIN_neigh,Clark_CNV_LOSS_neigh,Clark_Phospho_DOWN_neigh,Clark_Phospho_UP_neigh,Clark_Prot_DOWN_neigh,Clark_Prot_UP_neigh,...,RCC_vs_PRAP_DOWN_neigh,RCC_vs_PRAP_UP_neigh,Treg_markers_neigh,Tumour_markers_neigh,UPGs_neigh,cDC1_markers_neigh,cDC2_markers_neigh,pDC_markers_neigh,DTGs_neigh,Others_neigh
,,,,,,,,,,,,,,,,,,,,,
ENSP00000000233,0.000000,0.014815,0.0,0.000000,0.014815,0.029630,0.074074,0.059259,0.051852,0.029630,...,0.029630,0.007407,0.000000,0.014815,0.103704,0.014815,0.000000,0.014815,0.007407,0.444444
ENSP00000000412,0.007353,0.014706,0.0,0.022059,0.007353,0.029412,0.058824,0.029412,0.044118,0.014706,...,0.007353,0.022059,0.000000,0.029412,0.117647,0.022059,0.014706,0.029412,0.000000,0.397059
ENSP00000001008,0.000000,0.021858,0.0,0.000000,0.038251,0.021858,0.038251,0.049180,0.038251,0.027322,...,0.016393,0.043716,0.005464,0.032787,0.114754,0.016393,0.021858,0.021858,0.060109,0.344262
ENSP00000001146,0.000000,0.022472,0.0,0.000000,0.011236,0.022472,0.011236,0.000000,0.112360,0.000000,...,0.022472,0.000000,0.000000,0.033708,0.033708,0.000000,0.000000,0.000000,0.033708,0.539326
ENSP00000002125,0.000000,0.000000,0.0,0.000000,0.011494,0.011494,0.034483,0.022989,0.482759,0.000000,...,0.080460,0.011494,0.000000,0.034483,0.022989,0.000000,0.000000,0.000000,0.000000,0.321839


In [8]:
# Average and minimum network shortest path distances

network_distances = pd.read_csv("../Feature_space_generation/outputs/network_distances.csv", index_col=0)
network_distances.index.name = ""
network_distances.drop(columns = ["Min_DTGs", "Min_Others"], inplace=True)

network_distances = network_distances.loc[common_ids]
network_distances.sort_index(inplace=True)
network_distances.head()

,Mean_B_cells_markers,Mean_CAGs,Mean_CD4_markers,Mean_CD8_markers,Mean_Clark_CNV_GAIN,Mean_Clark_CNV_LOSS,Mean_Clark_Phospho_DOWN,Mean_Clark_Phospho_UP,Mean_Clark_Prot_DOWN,Mean_Clark_Prot_UP,...,Min_RCC_TFs_DOWN,Min_RCC_TFs_UP,Min_RCC_vs_PRAP_DOWN,Min_RCC_vs_PRAP_UP,Min_Treg_markers,Min_Tumour_markers,Min_UPGs,Min_cDC1_markers,Min_cDC2_markers,Min_pDC_markers
,,,,,,,,,,,,,,,,,,,,,
ENSP00000000233,2.513889,2.281046,2.176471,2.478261,2.664384,2.550117,2.439502,2.338710,2.553867,2.387097,...,2,2,1,1,2,1,1,1,2,1
ENSP00000000412,2.250000,2.320261,2.117647,2.152174,2.680365,2.566434,2.503559,2.459677,2.582873,2.381232,...,2,2,1,1,2,1,1,1,1,1
ENSP00000001008,2.194444,2.150327,2.176471,2.282609,2.515982,2.438228,2.361210,2.254032,2.383978,2.199413,...,2,1,1,1,1,1,1,1,1,1
ENSP00000001146,2.888889,2.568627,2.882353,2.847826,2.901826,2.850816,2.733096,2.762097,2.693370,2.771261,...,2,2,1,2,2,1,1,2,2,2
ENSP00000002125,2.902778,2.699346,2.764706,2.847826,2.963470,2.813520,2.768683,2.808468,2.533149,2.841642,...,2,2,1,1,2,1,1,2,2,2


In [9]:
print(DepMap_gene.shape)
print(DepMap_neigh.shape)
print(centrality_scores.shape)
print(network_neighbourhood.shape)
print(network_distances.shape)

(17880, 16)
(17880, 16)
(17880, 4)
(17880, 35)
(17880, 68)


In [10]:
# Merge

gene_feature_dataset = pd.concat([DepMap_gene, DepMap_neigh, centrality_scores, network_neighbourhood, network_distances], axis=1, join="inner")
#gene_feature_dataset.to_csv("gene_feature_dataset.csv")
print(gene_feature_dataset.shape)
gene_feature_dataset.head()

(17880, 139)


,ACH_000159,ACH_000246,ACH_000250,ACH_000262,ACH_000272,ACH_000313,ACH_000411,ACH_000433,ACH_000459,ACH_000600,...,Min_RCC_TFs_DOWN,Min_RCC_TFs_UP,Min_RCC_vs_PRAP_DOWN,Min_RCC_vs_PRAP_UP,Min_Treg_markers,Min_Tumour_markers,Min_UPGs,Min_cDC1_markers,Min_cDC2_markers,Min_pDC_markers
,,,,,,,,,,,,,,,,,,,,,
ENSP00000000233,0.014338,0.014540,0.012414,0.008893,0.010900,0.019440,0.012632,0.080683,0.014468,0.007162,...,2,2,1,1,2,1,1,1,2,1
ENSP00000000412,0.022938,0.002169,0.011734,0.015452,0.007020,0.097726,0.009703,0.016714,0.018487,0.013308,...,2,2,1,1,2,1,1,1,1,1
ENSP00000001008,0.066159,0.016908,0.025620,0.070442,0.014907,0.034068,0.035755,0.013399,0.016162,0.012158,...,2,1,1,1,1,1,1,1,1,1
ENSP00000001146,0.019598,0.005997,0.021080,0.010145,0.011347,0.078015,0.004898,0.072882,0.069126,0.017841,...,2,2,1,2,2,1,1,2,2,2
ENSP00000002125,0.074261,0.307553,0.095555,0.039562,0.078746,0.038826,0.090187,0.057175,0.028967,0.041280,...,2,2,1,1,2,1,1,2,2,2


In [11]:
# Import class1

known_DTGs = pd.read_csv("../data/DTGs/DTGs_class1_202Prot.csv")
known_DTGs = known_DTGs["x"].tolist()
known_DTGs = [DTG for DTG in known_DTGs if DTG in gene_feature_dataset.index]
known_DTGs = set(known_DTGs)
print(len(known_DTGs))

199


In [12]:
### Split data for model training and testing ###

In [13]:
# Set aside 20% of known drug target genes for testing

random.seed(7)
test_positive_examples = sample(known_DTGs, 40)

test_DTGs = open(os.path.join(out_dir, "testing_class1.txt"), "w")
for DTG in test_positive_examples:
    test_DTGs.write(DTG)
    test_DTGs.write("\n")
test_DTGs.close()

positive_examples = list(set(known_DTGs).difference(set(test_positive_examples)))
#set(positive_examples + test_positive_examples) == set(known_DTGs)

training_DTGs = open(os.path.join(out_dir, "training_class1.txt"), "w")
for DTG in positive_examples:
    training_DTGs.write(DTG)
    training_DTGs.write("\n")
training_DTGs.close()

/var/folders/vz/0nnw98hn1ndb6w2gt3rx780m0000gn/T/ipykernel_23564/3905306733.py:4: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  test_positive_examples = sample(known_DTGs, 40)


In [14]:
# Set aside 20% of class 0 genes for testing

all_genes = gene_feature_dataset.index.values.tolist()
all_genes_file = open(os.path.join(out_dir, "all_genes.txt"), "w")
for gene in all_genes:
    all_genes_file.write(gene)
    all_genes_file.write("\n")
all_genes_file.close()

random.seed(9)
other_genes = list(set(all_genes).difference(set(known_DTGs)))
other_genes = shuffle(other_genes)

random.seed(7)
test_negative_examples = sample(other_genes, 3530)
test_neg = open(os.path.join(out_dir, "testing_class0.txt"), "w")
for negative in test_negative_examples:
    test_neg.write(negative)
    test_neg.write("\n")
test_neg.close()

negative_genes = list(set(other_genes).difference(set(test_negative_examples)))
# print(set(negative_genes + test_negative_examples) == set(other_genes))
training_neg = open(os.path.join(out_dir, "training_class0.txt"), "w")
for negative in negative_genes:
    training_neg.write(negative)
    training_neg.write("\n")
training_neg.close()

In [15]:
positive_examples_in_all_genes = list(set(all_genes).intersection(set(positive_examples)))
num_positive_examples = len(positive_examples_in_all_genes)

print("Number of known DTGs:", len(known_DTGs)) #199
print("Number of class1 examples for training/cv:", num_positive_examples) #159
print("Number of class1 examples for testing:", len(test_positive_examples)) #40
print("Number of class0 genes:", len(other_genes)) #17681
print("Number of class0 genes for training/cv:", len(negative_genes)) #14151
print("Number of class0 genes for testing:", len(test_negative_examples)) #3530
print("Number of draws of class0 genes for balanced sampling:", len(negative_genes) / num_positive_examples) #89.0

Number of known DTGs: 199
Number of class1 examples for training/cv: 159
Number of class1 examples for testing: 40
Number of class0 genes: 17681
Number of class0 genes for training/cv: 14151
Number of class0 genes for testing: 3530
Number of draws of class0 genes for balanced sampling: 89.0


In [16]:
### ML training ###

In [17]:
# Define function for drawing negative examples chuncks from list of genes
def negative_sample_draw(gene_list, l = num_positive_examples, n=0):
    """get the nth chunck of negative examples"""
    return(gene_list[n*l:n*l+l])

selected_dataset = gene_feature_dataset.copy()
selected_dataset.to_csv(os.path.join(out_dir, 'selected_dataset.tsv'), sep="\t", index=True, header=True)
print(selected_dataset.shape)

(17880, 139)


In [18]:
#Define hyper-parameter tuning functions

def SVM_tuning(n, n_dir, X_train, y_train):
    scores = ['accuracy'] 
    
    grid_param_svm = [{'kernel': ['rbf'], 'gamma': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1], 'C': [0.1, 0.5, 1, 5, 10]},
                      {'kernel': ['linear'], 'C': [0.1, 0.5, 1, 5, 10]}]

    svm_tuning_info = open(os.path.join(n_dir, f'SVM_tuning_n{n}.txt'), "w")
    for score in scores:
        svm_tuning = GridSearchCV(SVC(random_state=3), grid_param_svm, cv=KFold(5, shuffle=True, random_state=3), scoring='%s' % score, n_jobs=6)
        svm_tuning.fit(X_train, y_train)

        print("# Tuning hyper-parameters for %s" % score, file=svm_tuning_info)
        print("Best parameters set found on training set:", file=svm_tuning_info)
        print(svm_tuning.best_params_, file=svm_tuning_info)
        print(file=svm_tuning_info)
        print("Grid scores on training set:", file=svm_tuning_info)
        means = svm_tuning.cv_results_['mean_test_score']
        stds = svm_tuning.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, svm_tuning.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params), file=svm_tuning_info)
        print(file=svm_tuning_info)

    print("Selected parameters:", file=svm_tuning_info)
    print(svm_tuning.best_params_, file=svm_tuning_info)
    svm_tuning_info.close()
    return(svm_tuning.best_params_)







def RF_tuning(n, n_dir, X_train, y_train):
    scores = ['accuracy'] 
    n_estimators = [1000, 1500, 2000, 2500]
    grid_param_rf = {'n_estimators': n_estimators}

    rf_tuning_info = open(os.path.join(n_dir, f'RF_tuning_n{n}.txt'), "w")
    for score in scores:
        rf_tuning = GridSearchCV(RandomForestClassifier(random_state=3), grid_param_rf, cv=KFold(5, shuffle=True, random_state=3), scoring='%s' % score, n_jobs=6)
        rf_tuning.fit(X_train, y_train)

        print("# Tuning hyper-parameters for %s" % score, file=rf_tuning_info)
        print("Best parameters set found on training set:", file=rf_tuning_info)
        print(rf_tuning.best_params_, file=rf_tuning_info)
        print(file=rf_tuning_info)
        print("Grid scores on training set:", file=rf_tuning_info)
        means = rf_tuning.cv_results_['mean_test_score']
        stds = rf_tuning.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, rf_tuning.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params), file=rf_tuning_info)
        print(file=rf_tuning_info)

    print("Selected parameters:", file=rf_tuning_info)
    print(rf_tuning.best_params_, file=rf_tuning_info)
    print(file=rf_tuning_info)

    max_features = ['sqrt', 'log2']
    max_depth = [5, 10, 20, 30, 40, 50, 60, 70, 80]
    max_depth.append(None)
    min_samples_split = [2, 5, 10, 15, 20, 25, 30]
    min_samples_leaf = [1, 2, 5, 10, 15, 20, 25, 30]
    grid_param_rf_random = {'max_features': max_features,
                            'max_depth': max_depth,
                            'min_samples_split': min_samples_split,
                            'min_samples_leaf': min_samples_leaf}
    for score in scores:
        rf_random = RandomizedSearchCV(estimator=RandomForestClassifier(n_estimators = rf_tuning.best_params_['n_estimators'], random_state=3), param_distributions=grid_param_rf_random, n_iter=100, cv=KFold(5, shuffle=True, random_state=3), verbose=0, scoring='%s' % score, n_jobs=6)
        rf_random.fit(X_train, y_train)

        print("# Randomized search for other hyper-parameters", file=rf_tuning_info)
        print("Best parameters set found on training set:", file=rf_tuning_info)
        print(rf_random.best_params_, file=rf_tuning_info)
        print(file=rf_tuning_info)
        print("Randomized search scores on training set:", file=rf_tuning_info)
        means = rf_random.cv_results_['mean_test_score']
        stds = rf_random.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, rf_random.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params), file=rf_tuning_info)
        print(file=rf_tuning_info)

    print("Selected parameters:", file=rf_tuning_info)
    print(rf_random.best_params_, file=rf_tuning_info)
    rf_tuning_info.close()
    
    rf_tuning.best_params_.update(rf_random.best_params_)
    return(rf_tuning.best_params_)

def GM_tuning(n, n_dir, X_train, y_train):
    scores = ['accuracy'] 
    n_estimators = [100, 250, 500, 750, 1000]
    learning_rate = [0.001, 0.005, 0.01, 0.05, 0.1, 0.2]
    grid_param_gbm = {'n_estimators': n_estimators,
                     'learning_rate': learning_rate}

    gbm_tuning_info = open(os.path.join(n_dir, f'GB_tuning_n{n}.txt'), "w")
    for score in scores:
        gbm_tuning = GridSearchCV(GradientBoostingClassifier(subsample=0.8, random_state=3), grid_param_gbm, cv=KFold(5, shuffle=True, random_state=3), scoring='%s' % score, n_jobs=6)
        gbm_tuning.fit(X_train, y_train)

        print("# Tuning hyper-parameters for %s" % score, file=gbm_tuning_info)
        print("Best parameters set found on training set:", file=gbm_tuning_info)
        print(gbm_tuning.best_params_, file=gbm_tuning_info)
        print(file=gbm_tuning_info)
        print("Grid scores on training set:", file=gbm_tuning_info)
        means = gbm_tuning.cv_results_['mean_test_score']
        stds = gbm_tuning.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, gbm_tuning.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params), file=gbm_tuning_info)
        print(file=gbm_tuning_info)

    print("Selected parameters:", file=gbm_tuning_info)
    print(gbm_tuning.best_params_, file=gbm_tuning_info)
    print(file=gbm_tuning_info)

    max_features = ['sqrt', 'log2']
    max_depth = [2, 3, 5, 8, 10, 20]
    max_depth.append(None)
    min_samples_split = [2, 5, 10, 15, 20, 25, 30]
    min_samples_leaf = [1, 2, 5, 10, 15, 20, 25, 30]
    grid_param_gmb_random = {'max_features': max_features,
                            'max_depth': max_depth,
                            'min_samples_split': min_samples_split,
                            'min_samples_leaf': min_samples_leaf}

    for score in scores:
        gbm_random = RandomizedSearchCV(estimator=GradientBoostingClassifier(n_estimators = gbm_tuning.best_params_['n_estimators'], learning_rate = gbm_tuning.best_params_['learning_rate'], subsample=0.8, random_state=3), param_distributions=grid_param_gmb_random, n_iter=100, cv=KFold(5, shuffle=True, random_state=3), verbose=0, scoring='%s' % score, n_jobs=6)
        gbm_random.fit(X_train, y_train)

        print("# Randomized search for other hyper-parameters", file=gbm_tuning_info)
        print("Best parameters set found on training set:", file=gbm_tuning_info)
        print(gbm_random.best_params_, file=gbm_tuning_info)
        print(file=gbm_tuning_info)
        print("Randomized search scores on training set:", file=gbm_tuning_info)
        means = gbm_random.cv_results_['mean_test_score']
        stds = gbm_random.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, gbm_random.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params), file=gbm_tuning_info)
        print(file=gbm_tuning_info)

    print("Selected parameters:", file=gbm_tuning_info)
    print(gbm_random.best_params_, file=gbm_tuning_info)
    gbm_tuning_info.close()
    
    gbm_tuning.best_params_.update(gbm_random.best_params_)
    return(gbm_tuning.best_params_)


In [19]:
#Model training

def model_training_n(n=0):
    n_dir = os.path.join(out_dir, f'draw_{n}')
    if not os.path.exists(n_dir):
        os.makedirs(n_dir)

    negative_examples = negative_sample_draw(negative_genes, n=n)
    training_examples = positive_examples_in_all_genes + negative_examples

    # Export negative class
    negative_example_file = open(os.path.join(n_dir, f'negative_class_n{n}.txt'), "w")
    for negative_example in negative_examples:
        negative_example_file.write(negative_example)
        negative_example_file.write("\n")
    negative_example_file.close()

    # Prepare training dataset
    train_val_dataset = selected_dataset.loc[selected_dataset.index.isin(training_examples)].copy()
    train_val_dataset['Targets'] = 0.0
    for target in train_val_dataset.index.to_list():
        if target in positive_examples_in_all_genes:
            train_val_dataset.loc[target, 'Targets'] = 1.0
    random.seed(4)
    train_val_dataset = shuffle(train_val_dataset)

    # Double-check that the training dataset does not contain labels
    train_val_data = train_val_dataset.iloc[:, 0:-1]
    for i in range(len(train_val_data.columns)):
        data = abs(train_val_data.iloc[:, i])
        if data.equals(train_val_dataset.iloc[:, -1]):
            raise Exception("Chunk n:", n, "target labels match feature:", i, train_val_data.columns[i], "nFeatures: ", train_val_data.shape[1])

    # Export training dataset
    feature_names = train_val_data.columns.to_list()
    train_val_data.to_csv(os.path.join(n_dir, f'training_data_n{n}.csv'), sep=",", index=True, header=True)


    
    # Create training data (X_train) and labels (y_train)
    X_train = train_val_dataset.iloc[:, 0:-1].values
    y_train = train_val_dataset.iloc[:, -1].values
    
     # Hyper-parameter tuning  
    svm_tuning = SVM_tuning(n=n, n_dir=n_dir, X_train=X_train, y_train=y_train)
    rf_tuning = RF_tuning(n=n, n_dir=n_dir, X_train=X_train, y_train=y_train)
    gm_tuning = GM_tuning(n=n, n_dir=n_dir, X_train=X_train, y_train=y_train)




    LR_model = LogisticRegression(solver='lbfgs', random_state=5, max_iter=500)
    
    if svm_tuning['kernel'] == 'rbf':
        SVM_model = SVC(C=svm_tuning['C'], 
                        gamma=svm_tuning['gamma'], 
                        kernel=svm_tuning['kernel'], 
                        probability=True,
                        random_state=5)
    elif svm_tuning['kernel'] == 'linear':
        SVM_model = SVC(C=svm_tuning['C'], 
                        kernel=svm_tuning['kernel'], 
                        probability=True, 
                        random_state=5)
    SVM_kernel = svm_tuning['kernel']
    RFC_model = RandomForestClassifier(n_estimators=rf_tuning['n_estimators'], 
                                       min_samples_split=rf_tuning['min_samples_split'], 
                                       min_samples_leaf=rf_tuning['min_samples_leaf'], 
                                       max_features=rf_tuning['max_features'], 
                                       max_depth=rf_tuning['max_depth'],
                                       random_state=5)
    GBM_model = GradientBoostingClassifier(n_estimators=gm_tuning['n_estimators'],
                                           learning_rate=gm_tuning['learning_rate'],
                                           subsample=0.8,
                                           min_samples_split=gm_tuning['min_samples_split'], 
                                           min_samples_leaf=gm_tuning['min_samples_leaf'], 
                                           max_features=gm_tuning['max_features'], 
                                           max_depth=gm_tuning['max_depth'],
                                           random_state=5)
    
    models = {'SVM': SVM_model, 'RF': RFC_model, 'GB': GBM_model, 'LR': LR_model} 

    # Define ensemble configurations
    ensemble_configs = {
        'Ensemble': list(models.keys())  # All models

    }

    # Initialize dictionaries to store scores
    model_scores = {name: {'Accuracy': [], 'Precision': [], 'Recall': [], 'F1': [], 'AUC': []} for name in models}
    ensemble_scores = {name: {'Accuracy': [], 'Precision': [], 'Recall': [], 'F1': [], 'AUC': []} for name in ensemble_configs}
    

    # Evaluate individual models
    for name, model in models.items():
        scores = cross_validate(
            model, X_train, y_train,
            cv=5,
            scoring=['accuracy', 'precision', 'recall', 'f1', 'roc_auc'],
            return_train_score=False, 
            n_jobs=6
        )
        model_scores[name]['Accuracy'] = scores['test_accuracy']
        model_scores[name]['Precision'] = scores['test_precision']
        model_scores[name]['Recall'] = scores['test_recall']
        model_scores[name]['F1'] = scores['test_f1']
        model_scores[name]['AUC'] = scores['test_roc_auc']

    # Evaluate ensemble models
    for ensemble_name, model_keys in ensemble_configs.items():
        estimators = [(key, models[key]) for key in model_keys]
        ensemble_model = VotingClassifier(estimators=estimators, voting='soft')
        scores = cross_validate(
            ensemble_model, X_train, y_train,
            cv=5,
            scoring=['accuracy', 'precision', 'recall', 'f1', 'roc_auc'],
            return_train_score=False, 
            n_jobs=6
        )
        ensemble_scores[ensemble_name]['Accuracy'] = scores['test_accuracy']
        ensemble_scores[ensemble_name]['Precision'] = scores['test_precision']
        ensemble_scores[ensemble_name]['Recall'] = scores['test_recall']
        ensemble_scores[ensemble_name]['F1'] = scores['test_f1']
        ensemble_scores[ensemble_name]['AUC'] = scores['test_roc_auc']

    # Aggregate and format the scores for each model
    final_scores = {}
    for name, scores in model_scores.items():
        final_scores[name] = {metric: f'{np.mean(values):.5f} (+/- {np.std(values) * 2:.5f})' for metric, values in scores.items()}

    # Aggregate and format the ensemble scores
    for ensemble_name, scores in ensemble_scores.items():
        final_scores[ensemble_name] = {metric: f'{np.mean(values):.5f} (+/- {np.std(values) * 2:.5f})' for metric, values in scores.items()}
    
    cv_scorings = pd.DataFrame(final_scores).transpose()
    cv_scorings.columns = ['Accuracy', 'Precision', 'Recall', 'F1', 'AUC']

    # Export cross-validation performance table
    cv_performance = pd.DataFrame(cv_scorings)
    cv_performance.to_csv(os.path.join(cv_dir, f'cv_performance_n{n}.csv'), index=True, index_label='Classifier')



        
     # Fit each model on the entire balanced training dataset and export
    for name, model in models.items():
  
        model.fit(X_train, y_train)
        pickle.dump(model, open(os.path.join(n_dir, f'{name}_m{n}.sav'), 'wb'))

        # Export feature importance rankings
        if name in ['RF', 'GB']:
            feature_ranking = pd.Series(model.feature_importances_, index=feature_names).sort_values(ascending=False)
            feature_ranking.to_csv(os.path.join(n_dir, f'{name}_f_ranking_n{n}.tsv'), index=True, sep=",", header=False)
            
        if name in ['LR']:
            coefficients = model.coef_[0]
            feature_importance_LR = pd.DataFrame({
                'Feature': feature_names,
                'Coefficient': coefficients,
                'Abs_Coefficient': np.abs(coefficients)
                })
            feature_importance_LR = feature_importance_LR.sort_values(by='Abs_Coefficient', ascending=False)
            feature_importance_LR.to_csv(os.path.join(n_dir, f'{name}_f_ranking_n{n}.tsv'), index=True, sep=",", header=False)


        if name in ['SVM']:
            if SVM_kernel == 'linear':
                coefficients = model.coef_[0]
                feature_importance_SVM = pd.DataFrame({
                    'Feature': feature_names,
                    'Coefficient': coefficients,
                    'Abs_Coefficient': np.abs(coefficients)
                })
                feature_importance_SVM = feature_importance_SVM.sort_values(by='Abs_Coefficient', ascending=False)
                feature_importance_SVM.to_csv(os.path.join(n_dir, f'{name}_f_ranking_n{n}.tsv'), index=True, sep=",", header=False)

            elif SVM_kernel == 'rbf':
                perm_result = permutation_importance(
                    model, X_train, y_train,
                    n_repeats=100,
                    random_state=42,
                    n_jobs=6)

                feature_importance_SVM = pd.DataFrame({
                    'Feature': feature_names,
                    'Importance': perm_result.importances_mean,
                    'Std': perm_result.importances_std
                    }).sort_values(by='Importance', ascending=False)
                feature_importance_SVM.to_csv(os.path.join(n_dir, f'{name}_f_ranking_n{n}.tsv'), index=True, sep=",", header=False)




        # Export model predictions on training set
        predicted_train = model.predict_proba(X_train) 
        np.savetxt(os.path.join(n_dir, f'{name}_pr{n}_train.csv'), predicted_train, delimiter=",", header="0,1")
        

        # Export model predictions on all genes
        random.seed(6)
        full_dataset = shuffle(selected_dataset)
        predicted_all = model.predict_proba(full_dataset) 
        predictions = pd.DataFrame(predicted_all, columns=['0', '1'], index=full_dataset.index.to_list())
        predictions.to_csv(os.path.join(n_dir, f'{name}_pr{n}_all_genes.csv'), index=True, sep=",", header=True)
        
    #Do the same for the ensemble models
    for ensemble_name, model_keys in ensemble_configs.items():
        estimators = [(key, models[key]) for key in model_keys]
        ensemble_model = VotingClassifier(estimators=estimators, voting='soft')
        ensemble_model.fit(X_train, y_train)
        pickle.dump(ensemble_model, open(os.path.join(n_dir, f'{ensemble_name}_m{n}.sav'), 'wb'))
        
        # Export predictions on training set
        predicted_train = ensemble_model.predict_proba(X_train)
        np.savetxt(os.path.join(n_dir, f'{ensemble_name}_pr{n}_train.csv'), predicted_train, delimiter=",", header="0,1")
        
        # Export predictions on all genes
        full_dataset = shuffle(selected_dataset)
        predicted_all = ensemble_model.predict_proba(full_dataset)
        predictions = pd.DataFrame(predicted_all, columns=['0', '1'], index=full_dataset.index.to_list())
        predictions.to_csv(os.path.join(n_dir, f'{ensemble_name}_pr{n}_all_genes.csv'), index=True, sep=",", header=True)

In [ ]:
for i in range(89):
    model_training_n(n=i)